### this notebook saves the metrics (static ones; and lss ones) for all (74) v1.7 dbs.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.7_-0.1cuts/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv_v1.7_-0.1cuts/'
os.makedirs(outdir, exist_ok=True)

#### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.65_nodither_nside256.csv
eg_footprint_stats_y3_limi25.25_nodither_nside256.csv
eg_footprint_stats_y6_limi25.619999999999997_nodither_nside256.csv
eg_footprint_stats_y10_limi25.9_nodither_nside256.csv
{'yr1_limi24.65': 'Y1 (i$>$24.65) ', 'yr3_limi25.25': 'Y3 (i$>$25.25) ', 'yr6_limi25.619999999999997': 'Y6 (i$>$25.619999999999997) ', 'yr10_limi25.9': 'Y10 (i$>$25.9) '}
dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.65
3 25.25
6 25.619999999999997
10 25.9


#### get the order of the dbs

In [5]:
#data['yr10_limi25.9'].dbname.values

In [6]:
# need to do this since input order is provided for only some of the dbs
df = pd.DataFrame(['rolling_scale0.9_nslice3_v1.7_10yrs',
 'rolling_scale0.2_nslice3_v1.7_10yrs',
 'rolling_scale1.0_nslice3_v1.7_10yrs',
 'rolling_scale0.9_nslice2_v1.7_10yrs',
 'rolling_scale0.6_nslice3_v1.7_10yrs',
 'rolling_scale0.6_nslice2_v1.7_10yrs',
 'rolling_scale0.2_nslice2_v1.7_10yrs',
 'rolling_scale0.8_nslice2_v1.7_10yrs',
 'rolling_scale0.4_nslice3_v1.7_10yrs',
 'rolling_scale0.4_nslice2_v1.7_10yrs',
 'rolling_scale0.8_nslice3_v1.7_10yrs',
 'rolling_scale1.0_nslice2_v1.7_10yrs',
 'footprint_1_v1.710yrs',
 'footprint_6_v1.710yrs',
 'footprint_4_v1.710yrs',
 'footprint_7_v1.710yrs',
 'footprint_0_v1.710yrs',
 'footprint_2_v1.710yrs',
 'footprint_5_v1.710yrs',
 'footprint_8_v1.710yrs',
 'footprint_3_v1.710yrs',
 'euclid_dither2_v1.7_10yrs',
 'euclid_dither4_v1.7_10yrs',
 'euclid_dither1_v1.7_10yrs',
 'euclid_dither5_v1.7_10yrs',
 'euclid_dither3_v1.7_10yrs',
 'rolling_nm_scale0.4_nslice2_v1.7_10yrs',
 'rolling_nm_scale0.4_nslice3_v1.7_10yrs',
 'rolling_nm_scale0.9_nslice2_v1.7_10yrs',
 'rolling_nm_scale0.8_nslice3_v1.7_10yrs',
 'rolling_nm_scale0.6_nslice2_v1.7_10yrs',
 'rolling_nm_scale1.0_nslice3_v1.7_10yrs',
 'rolling_nm_scale0.8_nslice2_v1.7_10yrs',
 'rolling_nm_scale0.2_nslice3_v1.7_10yrs',
 'rolling_nm_scale0.6_nslice3_v1.7_10yrs',
 'rolling_nm_scale1.0_nslice2_v1.7_10yrs',
 'rolling_nm_scale0.2_nslice2_v1.7_10yrs',
 'rolling_nm_scale0.9_nslice3_v1.7_10yrs',
 'pair_times_33_v1.7_10yrs',
 'pair_times_44_v1.7_10yrs',
 'pair_times_11_v1.7_10yrs',
 'pair_times_22_v1.7_10yrs',
 'pair_times_55_v1.7_10yrs',
 'twi_neo_pattern6_v1.7_10yrs',
 'twi_neo_pattern5_v1.7_10yrs',
 'twi_neo_pattern3_v1.7_10yrs',
 'twi_neo_pattern2_v1.7_10yrs',
 'twi_neo_pattern1_v1.7_10yrs',
 'twi_neo_pattern4_v1.7_10yrs',
 'twi_neo_pattern7_v1.7_10yrs',
 'u_long_ms_30_v1.7_10yrs',
 'u_long_ms_60_v1.7_10yrs',
 'u_long_ms_40_v1.7_10yrs',
 'u_long_ms_50_v1.7_10yrs',
 'twi_pairs_repeat_v1.7_10yrs',
 'twi_pairs_mixed_v1.7_10yrs',
 'twi_pairs_mixed_repeat_v1.7_10yrs',
 'twi_pairs_v1.7_10yrs',
 'baseline_nexp1_v1.7_10yrs',
 'baseline_nexp2_v1.7_10yrs',
 'ddf_dither2.00_v1.7_10yrs',
 'ddf_dither0.70_v1.7_10yrs',
 'ddf_dither0.00_v1.7_10yrs',
 'ddf_dither0.05_v1.7_10yrs',
 'ddf_dither0.10_v1.7_10yrs',
 'ddf_dither1.50_v1.7_10yrs',
 'ddf_dither1.00_v1.7_10yrs',
 'ddf_dither0.30_v1.7_10yrs',
 'cadence_drive_gl200v1.7_10yrs',
 'cadence_drive_gl100_gcbv1.7_10yrs',
 'cadence_drive_gl100v1.7_10yrs',
 'cadence_drive_gl30v1.7_10yrs',
 'cadence_drive_gl200_gcbv1.7_10yrs',
 'cadence_drive_gl30_gcbv1.7_10yrs']
                  , columns=['name'])
df.to_csv('%s/given_order_v1.7.csv' % outdir, index=False)

In [7]:
df_in_order = pd.read_csv('%s/given_order_v1.7.csv' % outdir, )
df_in_order

,name
0,rolling_scale0.9_nslice3_v1.7_10yrs
1,rolling_scale0.2_nslice3_v1.7_10yrs
2,rolling_scale1.0_nslice3_v1.7_10yrs
3,rolling_scale0.9_nslice2_v1.7_10yrs
4,rolling_scale0.6_nslice3_v1.7_10yrs
...,...
69,cadence_drive_gl100_gcbv1.7_10yrs
70,cadence_drive_gl100v1.7_10yrs
71,cadence_drive_gl30v1.7_10yrs
72,cadence_drive_gl200_gcbv1.7_10yrs


In [8]:
# check some things:
len( list( df_in_order['name'] ) ), len( np.unique( df_in_order['name'] ) )

(74, 74)

In [9]:
yr_key = [f for f in data.keys() if f.__contains__('yr1_')][0]
yr_key

'yr1_limi24.65'

#### find the indices to get the order

In [10]:
# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( data[yr_key]['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[12 16 18 21 22 23 36 37 38 46 53 70  6  8 13 14 17 26 31 52 71  1  2  4
  7  9  5 19 25 27 30 45 48 49 54 55 67 73 11 15 40 47 65  3 10 39 43 50
 51 66 20 32 41 68  0 28 42 64 33 69 56 57 58 59 60 61 62 63 24 29 34 35
 44 72]


In [11]:
# check we have the right indices
print( data[yr_key]['dbname'].values[28],  df_in_order['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

twi_pairs_mixed_v1.7_10yrs rolling_scale0.9_nslice3_v1.7_10yrs
74 74


#### save the data in the order in which to save things

In [12]:
data.keys()

dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])

In [13]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             df_in_order[colname] = data[yr_lim_tag][colname].values[ind]

In [14]:
df_in_order

,name,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,rolling_scale0.9_nslice3_v1.7_10yrs,14779.75,25.03,0.17,15237.64,25.65,0.13,15143.17,25.99,0.12,15152.50,26.25,0.11
1,rolling_scale0.2_nslice3_v1.7_10yrs,14779.75,25.03,0.17,15298.96,25.66,0.13,15184.45,25.99,0.12,15167.56,26.26,0.12
2,rolling_scale1.0_nslice3_v1.7_10yrs,14779.75,25.03,0.17,14946.19,25.65,0.13,15061.18,25.98,0.12,15048.59,26.24,0.11
3,rolling_scale0.9_nslice2_v1.7_10yrs,14779.75,25.03,0.17,15243.41,25.64,0.13,15125.33,25.98,0.12,15080.17,26.24,0.12
4,rolling_scale0.6_nslice3_v1.7_10yrs,14779.75,25.03,0.17,15224.84,25.65,0.13,15124.81,25.99,0.12,15098.42,26.25,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,14639.17,25.02,0.16,15250.07,25.66,0.13,15193.31,26.01,0.12,15201.71,26.27,0.11
70,cadence_drive_gl100v1.7_10yrs,14618.87,25.01,0.16,15245.93,25.66,0.13,15185.60,26.00,0.12,15188.75,26.27,0.11
71,cadence_drive_gl30v1.7_10yrs,14617.45,25.03,0.16,15232.50,25.67,0.13,15186.23,26.01,0.12,15191.95,26.27,0.11
72,cadence_drive_gl200_gcbv1.7_10yrs,13883.60,24.99,0.17,15061.02,25.62,0.13,15011.50,25.97,0.12,15003.69,26.24,0.12


In [15]:
cols = df_in_order.columns.tolist()
cols

['name',
 'Y1 effective survey area ',
 'median Y1 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'Y3 effective survey area ',
 'median Y3 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'Y6 effective survey area ',
 'median Y6 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'Y10 effective survey area ',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area']

#### realized that there's too much finagling when sharing data so its best to reorder columns to make sure the order of the columns is the same.

In [16]:
new_cols = ['name',
            'median Y1 $i$-band coadded depth in effective survey area',
            'median Y3 $i$-band coadded depth in effective survey area',
            'median Y6 $i$-band coadded depth in effective survey area',
            'median Y10 $i$-band coadded depth in effective survey area',
            'std in Y1 $i$-band coadded depth in effective survey area',
            'std in Y3 $i$-band coadded depth in effective survey area',
            'std in Y6 $i$-band coadded depth in effective survey area',
            'std in Y10 $i$-band coadded depth in effective survey area',
            'Y1 effective survey area ',
            'Y3 effective survey area ',
            'Y6 effective survey area ',
            'Y10 effective survey area ',]
new_cols

['name',
 'median Y1 $i$-band coadded depth in effective survey area',
 'median Y3 $i$-band coadded depth in effective survey area',
 'median Y6 $i$-band coadded depth in effective survey area',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area',
 'Y1 effective survey area ',
 'Y3 effective survey area ',
 'Y6 effective survey area ',
 'Y10 effective survey area ']

In [17]:
df_in_order = df_in_order[new_cols]
df_in_order

,name,median Y1 $i$-band coadded depth in effective survey area,median Y3 $i$-band coadded depth in effective survey area,median Y6 $i$-band coadded depth in effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area,Y1 effective survey area,Y3 effective survey area,Y6 effective survey area,Y10 effective survey area
0,rolling_scale0.9_nslice3_v1.7_10yrs,25.03,25.65,25.99,26.25,0.17,0.13,0.12,0.11,14779.75,15237.64,15143.17,15152.50
1,rolling_scale0.2_nslice3_v1.7_10yrs,25.03,25.66,25.99,26.26,0.17,0.13,0.12,0.12,14779.75,15298.96,15184.45,15167.56
2,rolling_scale1.0_nslice3_v1.7_10yrs,25.03,25.65,25.98,26.24,0.17,0.13,0.12,0.11,14779.75,14946.19,15061.18,15048.59
3,rolling_scale0.9_nslice2_v1.7_10yrs,25.03,25.64,25.98,26.24,0.17,0.13,0.12,0.12,14779.75,15243.41,15125.33,15080.17
4,rolling_scale0.6_nslice3_v1.7_10yrs,25.03,25.65,25.99,26.25,0.17,0.13,0.12,0.12,14779.75,15224.84,15124.81,15098.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,25.02,25.66,26.01,26.27,0.16,0.13,0.12,0.11,14639.17,15250.07,15193.31,15201.71
70,cadence_drive_gl100v1.7_10yrs,25.01,25.66,26.00,26.27,0.16,0.13,0.12,0.11,14618.87,15245.93,15185.60,15188.75
71,cadence_drive_gl30v1.7_10yrs,25.03,25.67,26.01,26.27,0.16,0.13,0.12,0.11,14617.45,15232.50,15186.23,15191.95
72,cadence_drive_gl200_gcbv1.7_10yrs,24.99,25.62,25.97,26.24,0.17,0.13,0.12,0.12,13883.60,15061.02,15011.50,15003.69


In [18]:
# save the metrics
fname = 'static_metrics.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)

#### now read and save lss-fom and ngal

In [19]:
# clean up the give_list dataframe
for key in df_in_order.keys():
    if key.__contains__('Y'):
        df_in_order = df_in_order.drop(key, axis=1)
        
df_in_order

,name
0,rolling_scale0.9_nslice3_v1.7_10yrs
1,rolling_scale0.2_nslice3_v1.7_10yrs
2,rolling_scale1.0_nslice3_v1.7_10yrs
3,rolling_scale0.9_nslice2_v1.7_10yrs
4,rolling_scale0.6_nslice3_v1.7_10yrs
...,...
69,cadence_drive_gl100_gcbv1.7_10yrs
70,cadence_drive_gl100v1.7_10yrs
71,cadence_drive_gl30v1.7_10yrs
72,cadence_drive_gl200_gcbv1.7_10yrs


#### read in the lss fom values

In [20]:
data_dir_base = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.7_-0.1cuts/lss_fom/'

In [21]:
data_dir = '%s/os_bias_overplots/' % data_dir_base
[f for f in os.listdir(data_dir) if f.__contains__('fomdata')] 

['2021-03-08_fomdata_Y1_i<23.95_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv',
 '2021-03-08_fomdata_Y10_i<25.2_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv']

In [22]:
fname = [f for f in os.listdir(data_dir) if f.__contains__('fomdata') if f.__contains__('_Y1_')][0]
print(fname)
data_file = '%s/%s' % (data_dir, fname)
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

df_in_order['Y1 lss fom'] = lss_fom['FoM'].values[ind]

2021-03-08_fomdata_Y1_i<23.95_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv
[ 6 12 16 19 20 21 35 36 37 46 55 65  3  4  8  9 14 24 30 54 68 11 26 42
 48 53  2 17 23 25 29 45 49 50 56 57 61 72  7 10 39 47 59  1  5 38 43 51
 52 60 18 31 40 62  0 27 41 58 32 64 13 15 63 66 67 69 70 73 22 28 33 34
 44 71]


In [23]:
fname = [f for f in os.listdir(data_dir) if f.__contains__('fomdata') if f.__contains__('_Y10_')][0]
print(fname)
data_file = '%s/%s' % (data_dir, fname)
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

df_in_order['Y10 lss fom'] = lss_fom['FoM'].values[ind]

2021-03-08_fomdata_Y10_i<25.2_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv
[ 6 12 16 19 20 21 35 36 37 46 55 65  3  4  8  9 14 24 30 54 68 11 26 42
 48 53  2 17 23 25 29 45 49 50 56 57 61 72  7 10 39 47 59  1  5 38 43 51
 52 60 18 31 40 62  0 27 41 58 32 64 13 15 63 66 67 69 70 73 22 28 33 34
 44 71]


In [24]:
df_in_order


,name,Y1 lss fom,Y10 lss fom
0,rolling_scale0.9_nslice3_v1.7_10yrs,0.904981,0.971725
1,rolling_scale0.2_nslice3_v1.7_10yrs,0.904981,0.973896
2,rolling_scale1.0_nslice3_v1.7_10yrs,0.904981,0.970231
3,rolling_scale0.9_nslice2_v1.7_10yrs,0.904981,0.971668
4,rolling_scale0.6_nslice3_v1.7_10yrs,0.904981,0.972016
...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,0.899427,0.975399
70,cadence_drive_gl100v1.7_10yrs,0.907829,0.975171
71,cadence_drive_gl30v1.7_10yrs,0.902464,0.975110
72,cadence_drive_gl200_gcbv1.7_10yrs,0.883126,0.968590


#### now read in the ngal values

In [25]:
ngal_data_dir = data_dir_base

ngal_data = {}
for yr in [1, 10]:
    ngal_data[yr] =  np.zeros(len(df_in_order.name))
    folders = [f for f in os.listdir( ngal_data_dir ) if f.__contains__('_Y%s_' % yr) and f.__contains__('i<') ]
    for i, folder in enumerate( folders ):
        out = pd.read_csv('%s/%s/ngal-final_total/ngal_NoDither.txt' % (ngal_data_dir, folder), header=None, delimiter=' ') 
        dbname, ngal = out[0].values[0], out[1].values[0]
        ind = np.where( df_in_order['name'].values == '%s' % dbname)[0]
        ngal_data[yr][ind] = ngal
    print('read in %s folders for Y%s' % (i+1, yr) )

read in 74 folders for Y1
read in 74 folders for Y10


In [26]:
for yr in ngal_data:
    df_in_order['Y%s Ngal (0.66<z<1.0)' % yr] = ngal_data[yr]

In [27]:
df_in_order

,name,Y1 lss fom,Y10 lss fom,Y1 Ngal (0.66<z<1.0),Y10 Ngal (0.66<z<1.0)
0,rolling_scale0.9_nslice3_v1.7_10yrs,0.904981,0.971725,431000000.0,942000000.0
1,rolling_scale0.2_nslice3_v1.7_10yrs,0.904981,0.973896,431000000.0,948000000.0
2,rolling_scale1.0_nslice3_v1.7_10yrs,0.904981,0.970231,431000000.0,940000000.0
3,rolling_scale0.9_nslice2_v1.7_10yrs,0.904981,0.971668,431000000.0,942000000.0
4,rolling_scale0.6_nslice3_v1.7_10yrs,0.904981,0.972016,431000000.0,943000000.0
...,...,...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,0.899427,0.975399,427000000.0,950000000.0
70,cadence_drive_gl100v1.7_10yrs,0.907829,0.975171,426000000.0,950000000.0
71,cadence_drive_gl30v1.7_10yrs,0.902464,0.975110,426000000.0,950000000.0
72,cadence_drive_gl200_gcbv1.7_10yrs,0.883126,0.968590,404000000.0,937000000.0


In [28]:
fname = 'lss_metrics.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)